In [8]:
import wandb
import pandas as pd
import numpy as np
import texttable as tt
import latextable

In [9]:
EXPERIMENT_GROUP = "exp-1-dataset-split-strategy"
BASELINE_SETUP = "random"
AVERAGE_EPOCHS = 5 # or None | must be odd

In [13]:
def get_best_epoch_metrics(run, metric_name, average_epochs=None):
    history = run.history(samples=10000)
    
    # Find the epoch with the best metric value
    best_epoch = history[metric_name].idxmax()
    best_metrics = history.loc[best_epoch]
    
    if average_epochs:
        # Find indices of valid (non-NaN) values
        valid_indices = history[metric_name].dropna().index
        
        # Locate the best epoch index in the valid indices
        best_index_position = valid_indices.get_loc(best_epoch)
        
        # Get the indices of the AVERAGE_EPOCHS epochs to be averaged
        idx_adjustment = (average_epochs - 1) // 2
        start_index = max(0, best_index_position - idx_adjustment)
        end_index = min(len(valid_indices), best_index_position + idx_adjustment + 1)
        selected_indices = valid_indices[start_index:end_index]
        
        # Average the metrics over the selected epochs
        averaged_metrics = history.loc[selected_indices].mean()
        
        print(f"Run {run.url} - Best epoch: {best_epoch} (averaged over {average_epochs}) - {metric_name}: {averaged_metrics[metric_name]}")
        return averaged_metrics, best_epoch
    else:
        print(f"Run {run.url} - Best epoch: {best_epoch} - {metric_name}: {best_metrics[metric_name]}")
        return best_metrics, best_epoch

def gather_metrics_for_experiment_group(experiment_group, average_epochs=False, has_weighted_accuracy=True):
    api = wandb.Api()

    # Fetch runs matching the experiment group
    runs = api.runs("crop-classification/messis", filters={"config.experiment_group": experiment_group})
    print(f"Found {len(runs)} runs for experiment group '{experiment_group}'")

    # Organize the runs by setup name
    setups = {}
    for run in runs:
        if 'name' not in run.config:
            if 'experiment_name' in run.config:
                setup_name = run.config['experiment_name'].split('-')[-1]
                print(f"Run {run.url} - Missing 'name' in config, getting it from 'experiment_name' instead: {setup_name}")
            else:
                print(f"Run {run.url} - Missing 'name' in config, skipping")
                continue
        else:
            setup_name = run.config['name']
        if setup_name not in setups:
            setups[setup_name] = {
                'runs': [],
                'metrics': {metric: [] for metric in [
                    'val_f1_tier1_majority',
                    'val_f1_tier2_majority',
                    'val_f1_tier3_majority',
                    'val_weighted_accuracy_tier1_majority' if has_weighted_accuracy else 'val_accuracy_tier1_majority',
                    'val_weighted_accuracy_tier2_majority' if has_weighted_accuracy else 'val_accuracy_tier2_majority',
                    'val_weighted_accuracy_tier3_majority' if has_weighted_accuracy else 'val_accuracy_tier3_majority',
                    'val_precision_tier1_majority',
                    'val_precision_tier2_majority',
                    'val_precision_tier3_majority',
                    'val_recall_tier1_majority',
                    'val_recall_tier2_majority',
                    'val_recall_tier3_majority',
                    'val_cohen_kappa_tier1_majority',
                    'val_cohen_kappa_tier2_majority',
                    'val_cohen_kappa_tier3_majority',
                ]},
                'best_steps': []
            }
        setups[setup_name]['runs'].append(run)

    # Print setup names with number of runs
    for setup_name, setup_data in setups.items():
        print(f"Setup '{setup_name}' - {len(setup_data['runs'])} runs")

    # Gather metrics for each setup
    for setup_name, setup_data in setups.items():
        for run in setup_data['runs']:
            best_metrics, best_epoch = get_best_epoch_metrics(run, 'val_f1_tier3_majority', average_epochs)
            setup_data['best_steps'].append(best_epoch)
            for metric in setup_data['metrics']:
                setup_data['metrics'][metric].append(best_metrics[metric])
    
    # Prepare data for DataFrame
    data = []
    for setup_name, setup_data in setups.items():
        metrics_summary = {
            'setup': setup_name,
            'num_runs': len(setup_data['runs']),
            'average_epochs': average_epochs,
            'run_handles': [run.id for run in setup_data['runs']],
            'best_steps': setup_data['best_steps'],
        }
        for metric, values in setup_data['metrics'].items():
            metrics_summary[f'{metric}_mean'] = np.mean(values)
            metrics_summary[f'{metric}_std'] = np.std(values)
        data.append(metrics_summary)
    
    metrics_df = pd.DataFrame(data)
    
    return metrics_df

# Run the function for a given experiment group
metrics_df = gather_metrics_for_experiment_group(EXPERIMENT_GROUP, average_epochs=AVERAGE_EPOCHS)

# Save the DataFrame to a CSV file
metrics_df.to_csv(f"experiment_results/eval_{EXPERIMENT_GROUP}.csv", index=False)

metrics_df

Found 22 runs for experiment group 'exp-1-dataset-split-strategy'
Run https://wandb.ai/crop-classification/messis/runs/tmmtb4ce - Missing 'name' in config, getting it from 'experiment_name' instead: random
Run https://wandb.ai/crop-classification/messis/runs/04lknbug - Missing 'name' in config, getting it from 'experiment_name' instead: random
Setup 'stratified-longer' - 1 runs
Setup 'stratified-long' - 1 runs
Setup 'sechidis' - 5 runs
Setup 'random' - 5 runs
Setup 'bands' - 5 runs
Setup 'stratified' - 5 runs
Run https://wandb.ai/crop-classification/messis/runs/islwn1g7 - Best epoch: 7434 (averaged over 5) - val_f1_tier3_majority: 0.18139490187168122
Run https://wandb.ai/crop-classification/messis/runs/1wx9c8fy - Best epoch: 2064 (averaged over 5) - val_f1_tier3_majority: 0.14408288151025772
Run https://wandb.ai/crop-classification/messis/runs/lfj8cwu4 - Best epoch: 1050 (averaged over 5) - val_f1_tier3_majority: 0.09199657291173935
Run https://wandb.ai/crop-classification/messis/runs/

,setup,num_runs,average_epochs,run_handles,best_steps,val_f1_tier1_majority_mean,val_f1_tier1_majority_std,val_f1_tier2_majority_mean,val_f1_tier2_majority_std,val_f1_tier3_majority_mean,...,val_recall_tier2_majority_mean,val_recall_tier2_majority_std,val_recall_tier3_majority_mean,val_recall_tier3_majority_std,val_cohen_kappa_tier1_majority_mean,val_cohen_kappa_tier1_majority_std,val_cohen_kappa_tier2_majority_mean,val_cohen_kappa_tier2_majority_std,val_cohen_kappa_tier3_majority_mean,val_cohen_kappa_tier3_majority_std
0,stratified-longer,1,5,[islwn1g7],[7434],0.381075,0.000000,0.267287,0.000000,0.181395,...,0.256731,0.000000,0.174624,0.000000,0.866521,0.000000,0.721650,0.000000,0.677414,0.000000
1,stratified-long,1,5,[1wx9c8fy],[2064],0.399910,0.000000,0.264600,0.000000,0.144083,...,0.261352,0.000000,0.150909,0.000000,0.852520,0.000000,0.702697,0.000000,0.648562,0.000000
2,sechidis,5,5,"[lfj8cwu4, hmm1uo25, a42lq9q5, kstftel1, eyy30...","[1050, 1021, 1122, 1103, 1109]",0.378196,0.031965,0.211349,0.014070,0.094142,...,0.214312,0.012374,0.102784,0.007650,0.810183,0.032170,0.629465,0.018179,0.558674,0.018702
3,random,5,5,"[tmmtb4ce, 04lknbug, qj4o4ati, 2zgnr3fw, mudkt...","[1187, 1498, 1326, 1247, 1065]",0.403428,0.057137,0.220351,0.019427,0.105953,...,0.226245,0.017065,0.113676,0.013620,0.816497,0.027777,0.645091,0.021135,0.575700,0.023932
4,bands,5,5,"[fyumjoqi, 4nu14ssi, lzkvcm2x, mc68cdlz, blfh8...","[740, 1392, 1520, 899, 869]",0.384699,0.061576,0.218879,0.028843,0.099383,...,0.228762,0.023233,0.110627,0.016543,0.789356,0.018374,0.615074,0.054720,0.538760,0.059415
5,stratified,5,5,"[av80hhcp, lf9khgx7, 8591uumk, f8oi85s8, vjyei...","[1119, 1073, 1056, 1201, 816]",0.378024,0.023070,0.208712,0.009237,0.089251,...,0.212239,0.006696,0.097254,0.006759,0.810463,0.014770,0.627513,0.011822,0.545880,0.020874


In [18]:
# Mapping for short metric names
metric_name_mapping = {
    'val_f1_tier1_majority': 'F1 $T_1$',
    'val_f1_tier2_majority': 'F1 $T_2$',
    'val_f1_tier3_majority': 'F1 $T_3$',
    'val_accuracy_tier1_majority': 'Acc. $T_1$',
    'val_accuracy_tier2_majority': 'Acc. $T_2$',
    'val_accuracy_tier3_majority': 'Acc. $T_3$',
    'val_weighted_accuracy_tier1_majority': 'W. Acc. $T_1$',
    'val_weighted_accuracy_tier2_majority': 'W. Acc. $T_2$',
    'val_weighted_accuracy_tier3_majority': 'W. Acc. $T_3$',
    'val_precision_tier1_majority': 'Precision $T_1$',
    'val_precision_tier2_majority': 'Precision $T_2$',
    'val_precision_tier3_majority': 'Precision $T_3$',
    'val_recall_tier1_majority': 'Recall $T_1$',
    'val_recall_tier2_majority': 'Recall $T_2$',
    'val_recall_tier3_majority': 'Recall $T_3$',
    'val_cohen_kappa_tier1_majority': 'Kappa $T_1$',
    'val_cohen_kappa_tier2_majority': 'Kappa $T_2$',
    'val_cohen_kappa_tier3_majority': 'Kappa $T_3$',
}

# Function to create a LaTeX table with bold best scores and underlined second-best scores
def create_latex_table(df, baseline_setup, first_n_metrics=None):
    metrics = list(metric_name_mapping.keys())
    if first_n_metrics:
        metrics = metrics[:first_n_metrics]
    
    # Determine the best and second-best scores for each metric
    best_scores = {metric: df[f'{metric}_mean'].max() for metric in metrics}
    second_best_scores = {
        metric: sorted(df[f'{metric}_mean'].unique(), reverse=True)[1] if len(df[f'{metric}_mean'].unique()) > 1 else None
        for metric in metrics
    }

    # Reorder the dataframe to have the baseline setup first
    df = df.set_index('setup')
    if baseline_setup in df.index:
        df = df.loc[[baseline_setup] + [idx for idx in df.index if idx != baseline_setup]].reset_index()
    else:
        df = df.reset_index()

    # Initialize the Texttable object
    table = tt.Texttable()
    
    # Define the header
    headers = ['Metric'] + [f"{'$\\dagger$ ' if row['setup'] == baseline_setup else ''}{setup} ({row['num_runs']}-fold)" 
                            for idx, row in df.iterrows() for setup in [row['setup']]]
    table.header(headers)
    
    # Set alignment for columns
    cols_align = ["c"] * len(headers)
    cols_align[0] = "l"
    table.set_cols_align(cols_align)
    
    # Populate the table with data
    for metric in metrics:
        row_data = [metric_name_mapping[metric]]
        for idx, row in df.iterrows():
            mean = row[f'{metric}_mean'] * 100
            std = row[f'{metric}_std'] * 100
            value = f'{mean:.1f}\\% ± {std:.1f}\\%'
            if mean == best_scores[metric] * 100:
                value = f'\\textbf{{{value}}}'
            elif second_best_scores[metric] and mean == second_best_scores[metric] * 100:
                value = f'\\underline{{{value}}}'
            row_data.append(value)
        table.add_row(row_data)
    
    # Generate the LaTeX table
    average_epochs_used = df['average_epochs'].iloc[0]
    caption_averaged =  f"averaged over {average_epochs_used} epochs" if average_epochs_used else ""
    latex_table = latextable.draw_latex(
        table, 
        caption=f"Results for experiment {EXPERIMENT_GROUP}. All metrics calculated on field majority during validation phase {caption_averaged}, with mean and std reported. Best scores bold, second best underlined, $\\dagger$ is baseline.", 
        label=f"tab:metrics_comparison_{EXPERIMENT_GROUP}", 
        use_booktabs=True,
        position="h"
    )

    # Add \resizebox to the LaTeX table
    latex_table = latex_table.replace('\\begin{center}', '\\begin{center}\\resizebox{\\textwidth}{!}{')
    latex_table = latex_table.replace('\\end{center}', '}\\end{center}')

    return latex_table

# Generate the LaTeX table
df = pd.read_csv(f"experiment_results/eval_{EXPERIMENT_GROUP}.csv")
df = df.sort_values(by='setup')

# Optional: Drop rows to exclude some setups
# df = df.drop([0, 1])

latex_table = create_latex_table(df, baseline_setup=BASELINE_SETUP, first_n_metrics=6)
print(latex_table)

\begin{table}[h]
	\begin{center}\resizebox{\textwidth}{!}{
		\begin{tabular}{lcccccc}
			\toprule
			Metric & $\dagger$ random (5-fold) & bands (5-fold) & sechidis (5-fold) & stratified (5-fold) & stratified-long (1-fold) & stratified-longer (1-fold) \\
			\midrule
			F1 $T_1$ & \textbf{40.3\% ± 5.7\%} & 38.5\% ± 6.2\% & 37.8\% ± 3.2\% & 37.8\% ± 2.3\% & \underline{40.0\% ± 0.0\%} & 38.1\% ± 0.0\% \\
			F1 $T_2$ & 22.0\% ± 1.9\% & 21.9\% ± 2.9\% & 21.1\% ± 1.4\% & 20.9\% ± 0.9\% & \underline{26.5\% ± 0.0\%} & \textbf{26.7\% ± 0.0\%} \\
			F1 $T_3$ & 10.6\% ± 1.6\% & 9.9\% ± 2.2\% & 9.4\% ± 0.9\% & 8.9\% ± 0.7\% & \underline{14.4\% ± 0.0\%} & \textbf{18.1\% ± 0.0\%} \\
			Acc. $T_1$ & \textbf{40.7\% ± 5.5\%} & 38.8\% ± 5.8\% & 36.9\% ± 2.8\% & 37.3\% ± 2.3\% & \underline{39.4\% ± 0.0\%} & 35.9\% ± 0.0\% \\
			Acc. $T_2$ & 22.6\% ± 1.7\% & 22.9\% ± 2.3\% & 21.4\% ± 1.2\% & 21.2\% ± 0.7\% & \textbf{26.1\% ± 0.0\%} & \underline{25.7\% ± 0.0\%} \\
			Acc. $T_3$ & 11.4\% ± 1.4\% & 11.1\% ± 1